# Práctica 1. Diferenciación de entidades.
## Autores: 
     Raúl Sánchez Martín
     Ignacio Arias Barra

In [1]:
import re, pprint, os, numpy
# -*- coding: utf-8 -*-
import nltk
from nltk import ngrams
import goslate
from nltk.collocations import *
import string
path_to_append = '/media/nacho/f8371289-0f00-4406-89db-d575f3cdb35e/Master/Trimestre 2/RIM/nltk_data'
# path_to_append = '/media/raul/Data/nltk_data'
path_to_append = '/home/raul/nltk_data'
nltk.data.path.append(path_to_append)
from sklearn.metrics.cluster import *
from sklearn.cluster import AgglomerativeClustering, KMeans, MiniBatchKMeans
from nltk.cluster import GAAClusterer
from sklearn.metrics.cluster import adjusted_rand_score
from nltk.corpus import stopwords
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import warnings
warnings.filterwarnings('ignore')
from nltk.tag import StanfordNERTagger
import os


## Funciones

### Código origen
Lectura de fichero:

In [55]:
def read_file(file):
    myfile = open(file,"r")
    data = ""
    lines = myfile.readlines()
    for line in lines:
        data = data + line
    myfile.close
    return data


Clustering:

In [56]:
def cluster_texts(texts, clustersNumber, distanceFunction, clusterMode):
    # Convierte textos en una coleccion
    # Load the list of texts into a TextCollection object.
    collection = nltk.TextCollection(texts)    
#     print("Created a collection of", len(collection), "terms.")
    
    # Get a list of unique terms
    unique_terms = list(set(collection))
#     print("Unique terms found: ", len(unique_terms))

    ### And here we actually call the function and create our array of vectors.
    # TF mide la frecuencia en los textos.
    # Mira de los terminos unicos, cuantas veces aparece en el documento. No mira cuantas veces aparece en la coleccion
    # Hay otras medidas, como TF-IDF que son mas precisas porque tambien miran cuantas veces aparece en la coleccion
    vectors = [numpy.array(TF(f,unique_terms, collection)) for f in texts]
    # print("Vectors created.")
    # print(vectors)
    
#     for vector in vectors:
#         print("Vector ", len(vector))

#     # initialize the clusterer
#     clusterer = GAAClusterer(clustersNumber)
#     clusters = clusterer.cluster(vectors, True)
    # Estas lineas siguientes comentadas es lo mismo pero con otra libreria, la llamada scikit-learn
    
    if clusterMode == "AgglomerativeClustering":
    
        clusterer = AgglomerativeClustering(n_clusters=clustersNumber,
                                         linkage="average", affinity=distanceFunction)
        clusters = clusterer.fit_predict(vectors)
    
    elif clusterMode == "KMeans":
    
        clusterer = KMeans(n_clusters=clustersNumber, random_state=0)
        clusters = clusterer.fit(vectors).predict(vectors)
        
    elif clusterMode == "MiniBatchKMeans":
        
        clusterer = MiniBatchKMeans(n_clusters=clustersNumber, random_state=0)
        clusters = clusterer.fit(vectors).predict(vectors)

    return clusters

Frecuencia de términos únicos en un documento

In [57]:
# Function to create a TF vector for one document. For each of
# our unique words, we have a feature which is the tf for that word
# in the current document
def TF(document, unique_terms, collection):
    word_tf = []
    for word in unique_terms:
        word_tf.append(collection.tf(word, document))
    return word_tf

### Nuevo código

*FUNCIÓN PARA DETECTAR EL IDIOMA DE UN TEXTO*

In [58]:
def get_language(possible_lan, text):
    # More info in: http://blog.alejandronolla.com/2013/05/15/detecting-text-language-with-python-and-nltk/
    languages_score = {}
    for language in possible_lan:
        stopwords_set = set(stopwords.words(language))
        words_set = set(text)
        common_elements = words_set.intersection(stopwords_set)
        languages_score[possible_lan[language]] = len(common_elements)
    return max(languages_score.items(), key=operator.itemgetter(1))[0]

*USO DE ENTIDADES I*

Se hará uso de las entidades reconocidas utilizando el método *ne_chunk* de la propia librería *nltk*. Simplemente se diferenciará entre nombres, adjetivos, adverbios, etc...

In [59]:
def get_named_entities_1(initial_document, selected_types):
    named_entities = list()
    selected_entities = list()
    try:
        for sentence in initial_document:
            tokenized_sentence = nltk.word_tokenize(sentence)
            # tagged_sentence = nltk.pos_tag(tokenized_sentence)
            tagged_sentence = nltk.pos_tag(tokenized_sentence, tagset= 'universal')
            named_ent = nltk.ne_chunk(tagged_sentence, binary=False)
            named_entities.append(named_ent)

        for element in named_entities:
            word = element.pos()[0][0][0]
            type_word = element.pos()[0][0][1]
            if type_word in selected_types:
                selected_entities.append(word)
        
        return selected_entities
    
    except Exception as e:
        print(str(e))
        
def get_named_entities_2(initial_document):
    named_entities = list()
    selected_entities = list()
    try:
        for sentence in initial_document:
            tokenized_sentence = nltk.word_tokenize(sentence)
            tagged_sentence = nltk.pos_tag(tokenized_sentence)
#             tagged_sentence = nltk.pos_tag(tokenized_sentence, tagset= 'universal')
            named_ent = nltk.ne_chunk(tagged_sentence, binary=False)
            named_entities.append(named_ent)
        
        for element in named_entities:
            if hasattr(element[0], 'label') and element[0].label:
                selected_entities.append(element[0].leaves()[0][0])
        
        return selected_entities 
    
    except Exception as e:
        print(str(e))

        
def get_named_ent_txts_1(raw_texts):
    named_ent_txts_1 = []
    for text in raw_texts:
        curr_named_ent = get_named_entities_1(text, types_included_1)
        text_to_append = nltk.Text(curr_named_ent)
        named_ent_txts_1.append(text_to_append)
    return named_ent_txts_1


def get_named_ent_txts_2(raw_texts):
    named_ent_txts_2 = []
    for text in raw_texts:
        curr_named_ent = get_named_entities_2(text)
        text_to_append = nltk.Text(curr_named_ent)
        named_ent_txts_2.append(text_to_append)
    return named_ent_txts_2


*USO DE ENTIDADES II (STANFORD)*

In [60]:


def get_entities_standorf(sample, types_named_entities):
    # Select the first classifier model
    stanford_classifier = os.environ.get('STANFORD_MODELS').split(':')[2]

    # Get the path for the StandorfNERTagger
    stanford_ner_path = os.environ.get('CLASSPATH').split(':')[0]
    
    st = StanfordNERTagger(stanford_classifier, stanford_ner_path, encoding='utf-8')
    result_named_entities = st.tag(sample.split())
    filtered_named_entities = []

    for item in result_named_entities:
        word, entity = item
        if entity in types_named_entities:
#             filtered_named_entities.append((word, entity))
            filtered_named_entities.append(word)
        
    return filtered_named_entities


def get_named_ent_txts_3(raw_texts, types_named_entities):
    named_ent_txts_3 = []
    for text in raw_texts:
        curr_named_ent = get_entities_standorf(text, types_named_entities)
        text_to_append = nltk.Text(curr_named_ent)
        named_ent_txts_3.append(text_to_append)
    return named_ent_txts_3


*ELIMINAR PALABRAS VACÍAS*

In [61]:
from nltk.corpus import stopwords


# # Abrimos el archivo.
# with open('./Thomas_Baker/005.txt', 'r', encoding="utf8") as f:
# # with open('./Thomas_Baker/036.txt', 'r', encoding="utf8") as f:
#     text = f.read()
# f.close()


def delete_stopwords(text,lan='en'):

    if lan == 'en':        
        # Obtenemos las sentencias del texto.
        sentences = nltk.sent_tokenize(text)
        # Obtenemos las "stopwords" del inglés.
        stop = set(stopwords.words('english'))

    elif lan == 'es':        
        # Elegimos el tokenizador para el español de la NLTK.
        spanish_tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')
        # Obtenemos las sentencias del texto.
        sentences = spanish_tokenizer.tokenize(text)        

        # Obtenemos las "stopwords" del español.
        stop = set(stopwords.words('spanish'))
      
    print("Frases originales: ")
    print(sentences)
    
    stopw = []
    for i in stop:
        stopw.append(i)
    print("Las stop words en lan = " + lan)
    print(stopw)
    print()

    # Eliminamos las palabras que coinciden con alguna "stopword".
    print("Frases procesadas sin stopwords: ")
    for sentence in sentences:
        non_stop_sentence = ""
        for word in sentence.lower().split():
            if word not in stop:
                non_stop_sentence = non_stop_sentence + word + " "
        # Imprimimos las sentencias procesadas.
        print(non_stop_sentence)
    return non_stop_sentence

def delete_words_from_text(text, words_to_delete):
    
    words_to_include = []
    words_to_delete = [word.lower() for word in words_to_delete]
    
    for word in text:
        if word.lower() not in words_to_delete:
            words_to_include.append(word)
            
    return words_to_include

def get_texts_no_stop_words(raw_texts):
    filtered_texts = []
    for text in raw_texts:
        language = {'en':'english', 'es':'spanish'}[get_language(possible_lan, text)]
        words_to_exclude = list(set(stopwords.words(language)))
        curr_filtered_text = delete_words_from_text(text, words_to_exclude)
        filtered_texts.append(nltk.Text(curr_filtered_text))
    return filtered_texts  

# espanol = delete_stopwords(text,lan='es')
# ingles = delete_stopwords(text)

*ELIMINAR EL NOMBRE DE LA PERSONA EN LOS TEXTOS*

In [62]:
# Eliminar las palabras ['Thomas', 'Baker'] de los textos

def get_texts_exclude_tomas_baker(raw_texts):
    filtered_texts = []
    for text in raw_texts:
        curr_filtered_text = delete_words_from_text(text, ['Thomas', 'Baker'])
        filtered_texts.append(nltk.Text(curr_filtered_text))
    return filtered_texts

*REPRESENTACIÓN DE NGRAMAS A NIVEL DE PALABRAS*

In [63]:
# Representación de ngramas

def get_ngram(raw_texts, ngramlimit):
    ngramlist=[]
    for text in raw_texts:
        ngram_text = nltk.ngrams(text, ngramlimit)
        ngramlist.append(nltk.Text(ngram_text))
    return ngramlist
                

*STEAMMING*

In [64]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

# # Abrimos el archivo.
# with open('./Thomas_Baker/005.txt', 'r', encoding="utf8") as f:
# with open('./Thomas_Baker/036.txt', 'r', encoding="utf8") as f:
#     text = f.read()
# f.close()

def stemming_original(text, lan='en'):
    stemmeds = []
    if lan == 'en':
        # Steamer ingles
        stemmer = PorterStemmer()        
    elif lan == 'es':
        # Stemer espanol        
        stemmer = SnowballStemmer("spanish")
    
    # Obtenemos los tokens del texto.
    tokens = nltk.word_tokenize(text)
        
    # Para cada token del texto obtenemos su raíz.
    for token in tokens:
        stemmed = stemmer.stem(token)
        stemmeds.append(stemmed)
    # Escribimos el resultado para compararlo con las palabras originales.
    print("Palabras originales: ")
    print(tokens)
    print("Palabras procesadas con Snowball: ")
    print(stemmeds)
    
    
def stemming(text, lan='en'):
    stemmeds = []
    if lan == 'en':
        # Steamer ingles
        stemmer = PorterStemmer()        
    elif lan == 'es':
        # Stemer espanol        
        stemmer = SnowballStemmer("spanish")
    
    # Obtenemos los tokens del texto.
    tokens = nltk.word_tokenize(text)
        
    # Para cada token del texto obtenemos su raíz.
    for token in tokens:
        stemmed = stemmer.stem(token)
        stemmeds.append(stemmed)
    # Escribimos el resultado para compararlo con las palabras originales.
    return stemmeds


possible_lan = {"english":"en", "spanish":"es"}
def get_stemmed_txts(raw_texts):
    stemmed_txts = []
    for text in raw_texts:
        language = get_language(possible_lan, text)
        stemmed_txt = stemming(text, language)
        text = nltk.Text(stemmed_txt)
        stemmed_txts.append(text)
    return stemmed_txts

def stemming_2(text, lan='en'):
    stemmeds = []
    if lan == 'en':
        # Steamer ingles
        stemmer = PorterStemmer()        
    elif lan == 'es':
        # Stemer espanol        
        stemmer = SnowballStemmer("spanish")
        
    # Para cada token del texto obtenemos su raíz.
    for word in text:
        stemmed = stemmer.stem(word)
        stemmeds.append(stemmed)
    # Escribimos el resultado para compararlo con las palabras originales.
    return stemmeds

def get_stemmed_txts_2(raw_texts):
    stemmed_txts = []
    for text in raw_texts:
        language = get_language(possible_lan, text)
        stemmed_txt = stemming_2(text, language)
        text = nltk.Text(stemmed_txt)
        stemmed_txts.append(text)
    return stemmed_txts

*ELIMINAR PALABRAS REPETIDAS*

In [65]:
# Pasamos el texto a minúsculas.
# Es necesario para no encontrar palabras repetidas que son la misma.
def delete_repWords_stopWords(raw_texts):    
    deleted_repeated = []
    for text in raw_texts:
        delete_repeated_texts = []
        text = [w.lower() for w in text]  
        unique_terms = list(set(text))
#         print("Número de palabras del texto: ",str(len(text)))
#         print("Tamaño del vocabulario filtrado: ", str(len(unique_terms)))
        deleted_repeated.append(nltk.Text(unique_terms))    
    return deleted_repeated

*LEMATIZAR LOS TEXTOS*

In [66]:
class SpanishLemmatizer():
    # Abrimos el fichero donde tenemos la información para cada palabra y lo cargamos en un diccionario.
    def __init__(self):
        with open('./Lemmatizer/lemmatization-es.txt', 'r', encoding="utf8") as f:
            self.lemma_dict = {}
            for line in f:
                if line.strip(): # Evitamos posibles líneas en blanco.
                    value, key = line.split(None, 1) # Nos quedamos con los valores clave y valor.
                                                     # None implica espacio en blanco.
                    key = key.rstrip() # Limpiamos la línea para evitar los caracteres de salto \n ó \r.
                    self.lemma_dict[key] = value
                    self.lemma_dict[value] = value  # Añadimos por si acaso también el valor como clave.

    # Obtenemos el lemma para la palabra solicitada si es que se dispone de él. En caso contrario devuelve la palabra.
    # Útil en los casos en los que no se haya aplicado un tratamiento previo del texto (stopwords y puntuación).
    def lemmatize(self,word):
        try:
            lemma = self.lemma_dict[word.lower()]
        except KeyError:
            lemma = word
        return lemma

def lemmatize_texts(raw_texts, possible_lan):
    nlemmas_texts = []
    for text in raw_texts:
        language = get_language(possible_lan, text)
        if language== 'en':
            # Seleccionamos el lematizador.
            wordnet_lemmatizer = WordNetLemmatizer()
            # Obtenemos los tokens de las sentencias.
#             tokens = nltk.word_tokenize(text)
            
            lemmatizeds = []
            nlemmas = []

            for token in text:                
                lemmatized = wordnet_lemmatizer.lemmatize(token)
                lemmatizeds.append(lemmatized)
                #print('token ',token)
                #print('lema ',lemmatized)
                # Obtenemos los lemmas consultando la base de datos de WordNet.
                list = wordnet.synsets(token)
                # Si encontramos alguna palabra relacionada obtenemos sus lemas y nos quedamos con el primero.
                if len(list) >= 1:
                    lemma = list[0].lemma_names('eng')
                    if len(lemma) > 1:
                        nlemmas.append(lemma[0])
                    else:
                        nlemmas.append((token))
                # En caso contrario simplemente introducimos en la solución la palabra actual.
            else:
                nlemmas.append(token)
            nlemmas_texts.append(nltk.Text(nlemmas))
        elif language== 'es':
            lemmatizer = SpanishLemmatizer()

            # Obtenemos los tokens del texto.
#             tokens = nltk.word_tokenize(text)            
            nlemmas = []
            lemmatizeds = []
            for token in text:                
                # Obtenemos los lemmas consultando el archivo de lemmas.
                lemmatized = lemmatizer.lemmatize(token)
                #print('token ',token)
                #print('lema ',lemmatized)
                lemmatizeds.append(lemmatized)
                # Obtenemos los lemmas consultando la base de datos de WordNet.
                list = wordnet.synsets(token, lang='spa')
                # Si encontramos alguna palabra relacionada obtenemos sus lemas y nos quedamos con el primero.
                if len(list) >= 1:
                    lemma = list[0].lemma_names('spa')
                    if len(lemma) >= 1:
                        nlemmas.append(lemma[0])
                    else:
                        nlemmas.append(token)
                # En caso contrario simplemente introducimos en la solución la palabra actual.
                else:
                    nlemmas.append(token)
            nlemmas_texts.append(nltk.Text(nlemmas))
        else:
            print('Lenguaje no reconocido')
    return nlemmas_texts

Ideas del propio enunciado de la práctica (muchas incluidas ya antes pero simmplemente para comprobar):

* Diferentes algoritmos de clustering
* Eliminar palabras vacías
* Stemming
* Lematización
* Reconocer entidades nombradas
* Eliminar palabras repetidas
* n-gramas
* Probar diferentes números de clusters


## Inicio programa

In [67]:
# Folder with all texts
folder = "Thomas_Baker"
types_named_entities = ["LOCATION", "PERSON", "ORGANIZATION"]
# Empty list to hold text documents.
raw_texts = []
raw_texts_2 = []
named_ent_txts_1 = []
types_included_1 = ['NOUN']

possible_lan = {"english":"en", "spanish":"es"}
clustering_modes = ["AgglomerativeClustering", "KMeans", "MiniBatchKMeans"]

listing = os.listdir(folder)
for file in sorted(listing):
    if file.endswith(".txt"):
        url = folder+"/"+file
        print(file)
        f = open(url,encoding="latin-1");
        raw = f.read()
        f.close()
        f2 = open(url, 'r', encoding="utf8")
        raw2 = f2.read()
        f2.close()
        raw_texts_2.append(raw2)
        tokens = nltk.word_tokenize(raw)
        text = nltk.Text(tokens)
        raw_texts.append(text)
        


print("Prepared ", len(raw_texts), " documents...")
print("They can be accessed using texts[0] - texts[" + str(len(raw_texts)-1) + "]")

001.txt
002.txt
005.txt
008.txt
009.txt
011.txt
015.txt
017.txt
020.txt
024.txt
027.txt
028.txt
036.txt
041.txt
047.txt
050.txt
056.txt
072.txt
075.txt
Prepared  19  documents...
They can be accessed using texts[0] - texts[18]


In [73]:
print('Using Stanford NER...')
stanford_ner_txts = get_named_ent_txts_3(raw_texts_2, types_named_entities)

print('Using Stanford NER removing Thomas Baker...')
stanford_ner_no_thomas_baker = get_texts_exclude_tomas_baker(stanford_ner_txts)

print("Removing non-stop words....")
texts_no_stop_words = get_texts_no_stop_words(raw_texts)

print("Removing Thomas Baker from the texts...")
texts_no_thomas_baker = get_texts_exclude_tomas_baker(raw_texts)

print("Getting text including only named entities according to criteria 1...")
named_ent_txts_1 = get_named_ent_txts_1(raw_texts)

print("Getting stemmed texts...")
stemmed_txts = get_stemmed_txts_2(raw_texts)

print("Getting no_tomas_stemmed...")
no_tomas_stemmed_txts = get_stemmed_txts_2(texts_no_thomas_baker)

print("Getting no_tomas_stemmed_no_stop...")
no_tomas_stemmed_no_stop_txts = get_texts_no_stop_words(no_tomas_stemmed_txts)

print("Getting no_tomas_stemmed_ent1...")
no_tomas_stemmed_ent1_txts = get_named_ent_txts_1(no_tomas_stemmed_txts)

print("Getting text including only named entities according to criteria 2...")
named_ent_txts_2 = get_named_ent_txts_2(raw_texts)

print("Getting text including only named entities according to criteria 2 and excluding the words 'Tomas' and 'Baker'")
named_ent_2_no_tomas_barker_txts = get_texts_exclude_tomas_baker(named_ent_txts_2)

print('Getting bigrams in texts')
bigrams_texts = get_ngram(raw_texts, 2)

print('Getting trigrams in texts')
trigrams_texts = get_ngram(raw_texts, 3)

print('Lemmatizing texts')
lemmatized_texts = lemmatize_texts(raw_texts, possible_lan)

print('Removing repeated words')
no_repeatedWords_noStopWords = delete_repWords_stopWords(raw_texts)


    
# named_ent_txts_2 = get_named_entities_2(raw_texts[0])
# for item in named_ent_txts_2:
#     if hasattr(item[0], 'label') and item[0].label:
#         print(item[0].leaves()[0][0])

# Similarity distance
distanceFunction ="cosine"
# distanceFunction = "euclidean"

reference =[0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 2, 0, 3, 3, 0, 1, 2, 0, 1]
print("reference: ", reference)

#------------------------------

# text_1 = "A football player is Raul Gonzalez"
# text_2 = "Raul Gonzalez is a football player"
# text_3 = "Data science is the future"
# text_4 = "The future is data science"

# text_1 = nltk.Text(nltk.word_tokenize(text_1))
# text_2 = nltk.Text(nltk.word_tokenize(text_2))
# text_3 = nltk.Text(nltk.word_tokenize(text_3))
# text_4 = nltk.Text(nltk.word_tokenize(text_4))



# new_texts = [text_1, text_2, text_3, text_4]

# collection_new = nltk.TextCollection(new_texts)

# print("Number of times Raul appears ", collection_new.tf("Raul", text_1))

# texts_new = [text_1, text_2, text_3, text_4]
# new_reference = [0, 0, 1, 1]
# new_test = cluster_texts(texts_new, 2, distanceFunction)
# print("New test ", new_test)
# print("New rand score ", adjusted_rand_score(new_reference, new_test))


Using Stanford NER...
Using Stanford NER removing Thomas Baker...
Removing non-stop words....
Removing Thomas Baker from the texts...
Getting text including only named entities according to criteria 1...
Getting stemmed texts...
Getting no_tomas_stemmed...
Getting no_tomas_stemmed_no_stop...
Getting no_tomas_stemmed_ent1...
Getting text including only named entities according to criteria 2...
Getting text including only named entities according to criteria 2 and excluding the words 'Tomas' and 'Baker'
Getting bigrams in texts
Getting trigrams in texts
Lemmatizing texts
Removing repeated words
reference:  [0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 2, 0, 3, 3, 0, 1, 2, 0, 1]


*CLUSTER FIJO CON 4 GRUPOS*

In [75]:
tested_models = {}
fix_grouping = 4
for cluster_mode in clustering_modes:
    tested_models[cluster_mode] = {}
    tested_models[cluster_mode]["primitive"] = cluster_texts(raw_texts,fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]["identity_analysis_1"] = cluster_texts(named_ent_txts_1,fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]["stemmed_txts"] = cluster_texts(stemmed_txts,fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]["no_tomas_baker"] = cluster_texts(texts_no_thomas_baker,fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]["no_stop_words"] = cluster_texts(texts_no_stop_words,fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]["no_tomas_stemmed"] = cluster_texts(no_tomas_stemmed_txts,fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]["no_tomas_stemmed_no_stop"] = cluster_texts(no_tomas_stemmed_no_stop_txts,fix_grouping,
                                                                            distanceFunction, cluster_mode)
    tested_models[cluster_mode]["no_tomas_stemmed_ent1"] = cluster_texts(no_tomas_stemmed_ent1_txts,fix_grouping,distanceFunction, 
                                                                         cluster_mode)
    tested_models[cluster_mode]["identity_analysis_2"] = cluster_texts(named_ent_txts_2,fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]["named_ent_2_no_tomas_barker"] = cluster_texts(named_ent_2_no_tomas_barker_txts,
                                                             fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]['bigrams'] = cluster_texts(bigrams_texts,fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]['trigrams'] = cluster_texts(trigrams_texts, fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]['lemmatized'] = cluster_texts(lemmatized_texts, fix_grouping,distanceFunction, cluster_mode)
    tested_models[cluster_mode]['no_repeated_words'] = cluster_texts(no_repeatedWords_noStopWords,fix_grouping, distanceFunction, cluster_mode)
    tested_models[cluster_mode]['stanford_ner_txts'] = cluster_texts(stanford_ner_txts,fix_grouping, 
                                                                     distanceFunction, cluster_mode)
    tested_models[cluster_mode]['stanford_ner_no_thomas_baker'] = cluster_texts(stanford_ner_no_thomas_baker,fix_grouping, 
                                                                     distanceFunction, cluster_mode)
    

# Evaluation
tested_models_scores = {}

for cluster_mode in tested_models:
    tested_models_scores[cluster_mode] = {}
    for model in tested_models[cluster_mode]:
        tested_models_scores[cluster_mode][model] = adjusted_rand_score(reference,tested_models[cluster_mode][model])
#     print("Model ", model, "; rand_score = ", adjusted_rand_score(reference,tested_models[model]))
    
for cluster_mode in tested_models_scores:
    print("**************************************")
    print("Getting results for the clustering mode ", cluster_mode)
    for model in sorted(tested_models_scores[cluster_mode].items(), key=operator.itemgetter(1), reverse=True):
        print("Model ", model[0], "; rand_score = ", model[1])
    print("########################################")
    print("########################################")

**************************************
Getting results for the clustering mode  KMeans
Model  identity_analysis_2 ; rand_score =  0.2304469273743017
Model  no_tomas_stemmed_no_stop ; rand_score =  0.17503392130257805
Model  primitive ; rand_score =  0.17503392130257805
Model  no_stop_words ; rand_score =  0.1492537313432836
Model  no_tomas_stemmed_ent1 ; rand_score =  0.14703968770331813
Model  no_tomas_baker ; rand_score =  0.12347354138398917
Model  lemmatized ; rand_score =  0.10494931425163982
Model  no_repeated_words ; rand_score =  0.0836012861736335
Model  named_ent_2_no_tomas_barker ; rand_score =  0.07255936675461744
Model  stanford_ner_no_thomas_baker ; rand_score =  0.05510752688172045
Model  stanford_ner_txts ; rand_score =  0.051395007342143924
Model  no_tomas_stemmed ; rand_score =  0.020352781546811426
Model  stemmed_txts ; rand_score =  0.020352781546811426
Model  identity_analysis_1 ; rand_score =  -0.05698778833107188
Model  trigrams ; rand_score =  -0.058949624866023

*DIFERENTE NÚMERO DE CLUSTERS*

In [76]:
tested_models = {}
top_cluster = 4
best_scores_all_clusters = {}
best_scores_Realcluster = {}
real_cluster_grouping = 4
init_scores = -999999
for cluster_mode in clustering_modes:
    best_scores_all_clusters[cluster_mode] = {}
    best_scores_Realcluster[cluster_mode] = {}

for cluster in range(1,top_cluster+1):
    fix_grouping = cluster
    print('CLASIFICATION WITH ' + str(fix_grouping) + ' CLUSTERS')
    for cluster_mode in clustering_modes:
        tested_models[cluster_mode] = {}        
        tested_models[cluster_mode]["primitive"] = cluster_texts(raw_texts,fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]["identity_analysis_1"] = cluster_texts(named_ent_txts_1,fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]["stemmed_txts"] = cluster_texts(stemmed_txts,fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]["no_tomas_baker"] = cluster_texts(texts_no_thomas_baker,fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]["no_stop_words"] = cluster_texts(texts_no_stop_words,fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]["no_tomas_stemmed"] = cluster_texts(no_tomas_stemmed_txts,fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]["no_tomas_stemmed_no_stop"] = cluster_texts(no_tomas_stemmed_no_stop_txts,fix_grouping,
                                                                                distanceFunction, cluster_mode)
        tested_models[cluster_mode]["no_tomas_stemmed_ent1"] = cluster_texts(no_tomas_stemmed_ent1_txts,fix_grouping,distanceFunction, 
                                                                             cluster_mode)
        tested_models[cluster_mode]["identity_analysis_2"] = cluster_texts(named_ent_txts_2,fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]["named_ent_2_no_tomas_barker"] = cluster_texts(named_ent_2_no_tomas_barker_txts,
                                                                 fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]['bigrams'] = cluster_texts(bigrams_texts,fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]['trigrams'] = cluster_texts(trigrams_texts, fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]['lemmatized'] = cluster_texts(lemmatized_texts, fix_grouping,distanceFunction, cluster_mode)
        tested_models[cluster_mode]['no_repeated_words'] = cluster_texts(no_repeatedWords_noStopWords,
                                                                         fix_grouping, distanceFunction, cluster_mode)
        
        
        tested_models[cluster_mode]['stanford_ner_txts'] = cluster_texts(stanford_ner_txts,fix_grouping, 
                                                                     distanceFunction, cluster_mode)
        tested_models[cluster_mode]['stanford_ner_no_thomas_baker'] = cluster_texts(stanford_ner_no_thomas_baker,fix_grouping, 
                                                                     distanceFunction, cluster_mode)
                

    # Evaluation
    tested_models_scores = {}

    for cluster_mode in tested_models:
        tested_models_scores[cluster_mode] = {}        
        for model in tested_models[cluster_mode]:            
            tested_models_scores[cluster_mode][model] = adjusted_rand_score(reference,tested_models[cluster_mode][model])
            
            # Calculate the max score for each model, each custer amount
            if model in best_scores_all_clusters[cluster_mode].keys():
                if best_scores_all_clusters[cluster_mode][model]['score'] <= tested_models_scores[cluster_mode][model]:
                    best_scores_all_clusters[cluster_mode][model]['cluster'] = cluster
                    best_scores_all_clusters[cluster_mode][model]['score'] = tested_models_scores[cluster_mode][model] 
            else:
                best_scores_all_clusters[cluster_mode][model] = {}
                best_scores_all_clusters[cluster_mode][model]['cluster'] = cluster
                best_scores_all_clusters[cluster_mode][model]['score'] = init_scores    
            
            if cluster == real_cluster_grouping:
                best_scores_Realcluster[cluster_mode][model] = {}
                best_scores_Realcluster[cluster_mode][model]['cluster'] = cluster
                best_scores_Realcluster[cluster_mode][model]['score'] = tested_models_scores[cluster_mode][model]              
    
    for cluster_mode in tested_models_scores:
        print("**************************************")
        print("Getting results for the clustering mode ", cluster_mode)
        for model in sorted(tested_models_scores[cluster_mode].items(), key=operator.itemgetter(1), reverse=True):
            print("Model ", model[0], "; rand_score = ", model[1])            
        print("########################################")
        print("########################################\n")

for cluster_mode in clustering_modes:
    print('/////////////////////////////////')
    print('CLUSTER ALGORITHM: ', cluster_mode)
    for model in best_scores_all_clusters[cluster_mode].keys():
                        
        print('*Model: "', model, '". Best cluster agroupation: ',
                  best_scores_all_clusters[cluster_mode][model]['cluster'],
                  ' clusters. Score: ', 
                  str(best_scores_all_clusters[cluster_mode][model]['score'])) 
        
        print('++Score  for the real cluster agroupation (',str(real_cluster_grouping),
              ') in model ', model, ' is ',
              str(best_scores_Realcluster[cluster_mode][model]['score'])) 
    print('/////////////////////////////////\n')

CLASIFICATION WITH 1 CLUSTERS
**************************************
Getting results for the clustering mode  KMeans
Model  no_tomas_stemmed_no_stop ; rand_score =  0.0
Model  identity_analysis_1 ; rand_score =  0.0
Model  no_tomas_stemmed ; rand_score =  0.0
Model  stemmed_txts ; rand_score =  0.0
Model  primitive ; rand_score =  0.0
Model  trigrams ; rand_score =  0.0
Model  bigrams ; rand_score =  0.0
Model  no_tomas_stemmed_ent1 ; rand_score =  0.0
Model  named_ent_2_no_tomas_barker ; rand_score =  0.0
Model  lemmatized ; rand_score =  0.0
Model  no_stop_words ; rand_score =  0.0
Model  no_tomas_baker ; rand_score =  0.0
Model  stanford_ner_no_thomas_baker ; rand_score =  0.0
Model  identity_analysis_2 ; rand_score =  0.0
Model  no_repeated_words ; rand_score =  0.0
Model  stanford_ner_txts ; rand_score =  0.0
########################################
########################################

**************************************
Getting results for the clustering mode  MiniBatchKM

In [12]:
# En la memoria se ha de justificar cada cambio propuesto.

# Se ha de comentar los resultados obtenidos.

# Siempre buscando una mejora en la clusterización